## Basic Algorithms: Fibonacci

Another form of recursion is tree recursion. Consider computing a fibonacci sequence, in which each number is the sum of the previous two, with the first two taken to be 0 and 1.

### Recursive Fibonacci. 


We write Fibonacci recursively with the first two numbers as base cases.

![](https://mitpress.mit.edu/sicp/full-text/book/ch1-Z-G-13.gif)

(from SICP)

Signature: `def fib_recursive(n)`

In [2]:
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [1, 2, 7, 13, 29, 33]:
    print(fib_recursive(i))

### Q1. 

What are the space and time complexities of this implementation? Hint: think aboutthe number of items in a binary tree and its depth.

The space complexity is $O(n)$ and the time compleity is $O(2^n)$. 

For space complexity, it is the depth of the binary tree n. On each level of the binary tree, we are calling the fib_recursive function, creating an additional frame that takes one unit of space. After running through the function and return a value, the frame does not exist anymore and releases the space before going to the next branch. Therefore, the maximum value of tree depth is n, which can be seen as the left most branch that goes from fib(n) to fib(1) by decrement of 1 each time in the graph above. That is, the space complexity is $O(n)$. 

For time complexity, it is the number of nodes that is included in the binary tree. At each node, we perform a constant number of operation, adding the value of its direct children, so the time needed at each node can be seen as constant time. To calculate the value on top of the tree, we need to tranverse through the entire tree, visiting each node exactly one time. To find the number of nodes, we notice that we break each node into two branches until we reach the bottom level. So the total number of node can be approximated by $1+2+4+\dots+2^n = \frac{1-2^n}{1-2} =2^n-1 = O(2^n)$. Therefore, the total time complexity is the number of nodes in the tree, which is $O(2^n)$. 


### Dynamic Programming and Iteration

From Skiena
>..**dynamic programming**, which typically removes one element from the problem, solves the smaller problem, and then uses the solution to this smaller problem to add back the element in the proper way. **Divide-and-conquer** instead splits the problem in (say) halves, solves each half, then stitches the pieces back together to form a full solution.

>Dynamic programming is a technique for efficiently implementing a recursive algorithm by storing partial results. The trick is seeing whether the naive recursive algorithm computes the same subproblems over and over and over again. If so, storing the answer for each subproblems in a table to look up instead of recompute can lead to an efficient algorithm. Start with a recursive algorithm or definition. Only once we have a correct recursive algorithm do we worry about speeding it up by using a results matrix. Dynamic programming is generally the right method for optimization problems on combinatorial objects that have an inherent left to right order among components. Left-to-right objects includes: character strings, rooted trees, polygons, and integer sequences.

### Q2.

Here is an implementation of Fibonacci using dynamic programming: they key is to notice that the recurrence we used can be put into an iterative form and just stored in an ever increasing array. What is the space and time complexity here? What if you were somehow able to save the array outside of the function when u calculate `fib(M)` and subsequently had to calculate `fib(N)`, where $N>M$.

In [14]:
def fib_iterative(n):
    fibs=[]
    fibs.append(0)
    fibs.append(1)
    for i in range(2, n):
         fibs.append(fibs[i-1]+fibs[i-2])
    return fibs[n-1]

In [17]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


By using dynamic programing, the space complexity is $O(n)$ and the time complexity is also $O(n)$. 

With the iterative method, we would need an array of size $n$ to store the calculated fib values, so the space complexity is $O(n)$, which is the same as before. For time complexity, now we only need to calculate fib(n) once for all n and store all previously calculated values so that we only need constant time accessing the value. Therefore, the time complexity is reduced to $O(n)$ by using dynamic programming. 

If we were able to save the array outside when calculating `fib(M)`, then when we had to calculate `fib(N)` for $N>M$, we only need additional $O(N-M)$ in space and $O(N-M)$ in time. It is still linear complexity in time and space. 

### Recursive Fibonacci with cacheing

### Q3. 

Use `cache` and `call_counter` as decorators on `fib_recursive` and print the fibonacci numbers for 7,13,29, 33. What order should these decorators be called to make sure `call_counter` gets the actual number of calls to `fib_recursive`?

We've written the `cache` decorator for you. You have to write the `call_counter` decorator which takes the function as argument and using a `count_dictionary` whose keys are function names, counts the number of times the function is called.

In [26]:
def cache(f):
    """a single argument function whose values may be cached"""
    cache = {}
    def memoized_func(x):
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    memoized_func.__name__ = f.__name__
    return memoized_func
    

In [32]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        def inner(*args, **kwargs):
            name = func.__name__
            if name in count_dictionary:
                count_dictionary[name] +=1
            else:
                count_dictionary[name] = 1
            return func(*args, **kwargs)
        return inner
    return call_counter_decorator


In [40]:
ccounter={}
@cache
@call_counter(ccounter)
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [1,2, 3, 7,13,29, 33]:
    print(i, fib_recursive(i), ccounter['fib_recursive'])

1 0 1
2 1 2
3 1 3
7 8 7
13 144 13
29 317811 29
33 2178309 33


We should call the `call_counter(ccounter)` first and then call the `cache()` decorator to make sure the call_counter decorator gets the actual number of calls to fib_ricursive. That is, call_counter is applied first and then we cache the results. 

### Q4.

What is the time and space complexity of the the memoized Fibonacci? HINT: assume evaluation happens left to right on sub-expressions, so that `fib_recursive(n-1)` side of the tree is evaluated first, and thus the tree is evaluated depth first, from left to right. What kind of pruning happens in the tree?

The time complexity is $O(n)$ and the space complexity is $O(n)$ for the memoized Fibonacci. 

By using the memoized Fibonacci, we are basically storing all previous values as in the dynamic programming method. After traversing down the tree from left sub-expressions, we computed all `fib_recursie(n-1)`. When we then try to call `fib_recursive(n-2)`, it should have been calculated once down the left branch, and thus we only need to access the value stored before. To visualize from the tree, we basically have a long branch to the left, and having one direct child to the right but not more branches further down the tree. 


### Q5.

Do you really need to store the entire array in the dynamic programming implementation? Isnt it enough to have only saved the previous two Fibonacci numbers? Implement such an algorithm in `fib_iterative2(n)`. What is its space and time complexity? How is the time complexity different from that of the previous iterative attempt?

In [21]:
#your code here
def fib_iterative2(n):
    fibs=[0,1]
    if n<2:
        return fibs[n-1]
    else:
        for i in range(2, n):
            fibs[0], fibs[1] = fibs[1], (fibs[0]+fibs[1])
        return fibs[1]


In [38]:
#your code here
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative2(i))

0
1
8
144
317811
2178309


We do not need to store the entire array in dynamic programming implementation, since we would only need the previous two values when calculating the next Fibonacci number. It would be enough if we only store the previous two. This new iterative method is implemented above as `fib_recursive2(n)`. 

The time complexity is still $O(n)$ but the space complexity is reduced to O(1) - a constant complexity. In this case, when we want to calculate `fib(n)`, we need to calculate each of the previous Fibbonacci numbers and store the previous two in the array. With each calculation is a constant time complexity by accessing the two values and add them up, the total time complexity would be a constant time times the $n$. Therefore, the time complexity is still $O(n)$, same as in the previous iterative method. The space complexity is reduced to constant $O(1)$. 


### Q6.

Write an algorithm for insertion sort.

![](https://camo.githubusercontent.com/8f6fedc10da579f13b22b949f6ad29255b6d721f/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f302f30662f496e73657274696f6e2d736f72742d6578616d706c652d33303070782e676966)

(from wikipedia)

The algorithm is also illustrated here: http://cs.armstrong.edu/liang/animation/web/InsertionSort.html and may be described thus:

Insertion sort is a method for sorting that starts with a single element (thus forming a trivially sorted list) and then incrementally inserts the remaining elements so that the list stays sorted.

Talk about the best, worst and average complexity of insertion sort. Use the A=[5,2,1,3,8,6,9] to show your code!

In [9]:
#your code here
def insertion_sort(input_list):
    for i in range(1,len(input_list)):
        j = i-1
        while((input_list[i] < input_list[j]) & (j >= 0)):
            input_list[i], input_list[j] = input_list[j], input_list[i]
            j -= 1
            i -=1
    return input_list


[1, 2, 3, 5, 6, 8, 9]

Best complexity: The best senario for insertion sort is when the original list is already sorted. In this case, when we iterate through the list, we only need to make one comparison with the element in front. Because the list is originally sorted, we will have the current value bigger than the previous one, and thus do not need to make more comparisons or swaps. The complexity in this best case is just to iterate through the list, which is $O(n)$. 

Worst complexity: The worst case for insertion sort is when the original listis in reverse order. In this cae, each element would need to compare and swap with all elements in front of it. That is, for the ith element, it will swap with i-1 elements, and i ranges from 1 to n. $\sum_{i=1}^{n} (i-1) = \frac{(n-1)n}{2} = O(n^2)$. Therefore, the worst complexuty is $O(n^2)$. 

Average complexity:


In [ ]:
sorted_list = insertion_sort([5,2,1,3,8,6,9])
print (sorted_list)

In [11]:
from IPython.display import HTML
HTML('''<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=def%20insertion_sort(input_list%29%3A%0A%20%20%20%20for%20i%20in%20range(1,len(input_list%29%29%3A%0A%20%20%20%20%20%20%20%20j%20%3D%20i-1%0A%20%20%20%20%20%20%20%20while((input_list%5Bi%5D%20%3C%20input_list%5Bj%5D%29%20%26%20(j%20%3E%3D%200%29%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20input_list%5Bi%5D,%20input_list%5Bj%5D%20%3D%20input_list%5Bj%5D,%20input_list%5Bi%5D%0A%20%20%20%20%20%20%20%20%20%20%20%20j%20-%3D%201%0A%20%20%20%20%20%20%20%20%20%20%20%20i%20-%3D1%0A%20%20%20%20return%20input_list%0Asorted_list%20%3D%20insertion_sort(%5B5,2,1,3,8,6,9%5D%29%0Aprint%20(sorted_list%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=false&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>''')